---
### Simplified System of linear equations solving for image $X$

Key terms:

$L$ = one-dimensional size for high-resolution image

$M$ = one-dimensional size for low-resolution image

$X$ = $L^2 \times 1$ column vector, represents the ideal image we are trying to recover via solving

$Y_i$ = $M^2 \times 1$ column vector, represents a decimated, downsampled, blurred, noisy image based on $X$

$N$ = the number of low-resolution images

$D$ = the decimation matrix operator of size $M^2 \times L^2$

$H$ = the blur matrix operator of size $L^2 \times L^2$

$S$ = the sharpening operator of size $L^2 \times L^2$, i.e., the laplacian operator

$w_i$ = a weighting scalar, a "confidence factor" for each $Y_i$ (eventually a diagonal matrix?)

$\beta$ = smoothing factor for controlling image sharpening

---

Objective: to solve for $X$ (i.e., $AX = B$) based on the following equation:

$\begin{bmatrix}
  \beta S^T S + (\sum\limits_{i=1}^N w_i) H^TD^TDH
\end{bmatrix}X = \sum\limits_{i=1}^N (w_iH^TD^TY_i)$

Note: In order to reconstruct a high-resolution image from $N$ low-resolution images, the following criteria must be satisfied: $L^2 \leq \min \left\{ \left(s^2 - 2 \right)M^2, NM^2\right\}$, where $s$ is the full support of the blurring kernel (i.e., point-spread function) (as per section III.B of [1]).

[1] M. Elad and A. Feuer, "Restoration of a single superresolution image from several blurred, noisy, and undersampled measured images," in IEEE Transactions on Image Processing, vol. 6, no. 12, pp. 1646-1658, Dec. 1997, doi: 10.1109/83.650118.

---

General information:

- Dataset: GLEAM Small
- Timesteps: 30
- Receivers: 512
- Channels: 1
- $L$ = 100 pixels
- $M$ = 50 pixels
- $N$ = 6 images (30 timesteps / 5 timesteps per image)
- $w_i$ = 1 for all images
- $\beta$ = 1 (I guess?)

Assumptions:
1. The blur kernel $H$ is uniform across all instances of $Y$
2. The decimatimation kernel $D$ is uniform across all instances of $Y$
3. The $w_i$ is uniform across all instances of $Y$, just a confidence of 1.0 ("full confidence")

---

#### To do:

- [x] Generate 6 dataset subsets, containing timesteps each (i.e., 30 time steps / 5 time steps per file)
- [x] Generate one point spread function per dataset subset, for analysis of different
- [x] Generate one point spread function for the whole dataset
- [x] Determine if there is a significant difference between a subset PSF and the full dataset PSF - this will be the blur kernel
- [ ] Determine if the PSF can be reduced (i.e., dont use the full PSF) - what effects might this have?
- [x] Generate an IDFT of the full dataset, $L^2$ pixels
- [x] Genetate an IDFT for each of the subsets, $M^2$ pixels
- [x] Come up with a formula for populating the decimation matrix
- [ ] Come up with a formula for populating the blur matrix 
- [ ] Determine if there is a formula for populating the product of the blur and decimation matrix, such that $H^TD^T = (HD)^T$
- [ ] take the average of optimal betas across support 20 to 47, use that for all supports, plot the error between solved x and true x using the averaged beta, and compare that against another plot which is the "optimal" beta vs support
- [ ] another thing to try, consider trimming the edge of solved x before taking norms and measuring RRMSE, as the convolution matrices are not padded and so convolution is essentially "incomplete" around the image edges. Probably only need to take say 10-15 pixels off all edges, since most of the PSF is noise aside from the center.
- [ ] Another thing to try is to produce a set of Y images which are only representative of a single source, like a PSF. Then I could run the SOLE algorithm to determine what the effects of solving are against the curve of a point source, by taking a middle slice of the image (like I do when inspecting the actual PSF). Could examine what happens when using different supports and beta terms to see which lines up best with the true curve.
 - [ ] I need to modify the decimation matrix function to support more than just 4 neighbours (or 1/2 decimation).
 - [ ] Modify the project so it is more like a library, where I can call my own functions in my desired order. This would make repeating experiments easier than one flat notebook.
 - [ ] Implement the laplacian of gaussian function, so that I can have a dynamically sized laplacian instead of a fixed 3x3 (note: requires another config to specify laplacian support size, and a sigma value to determine curve spread, or are they roughly the same type of thing?)
 - [ ] Try running the algorithm with the averaged beta, using different sized LoGs and different PSF supports to see what happens re: rrmse
 - [ ] Figure out how I can get the sigma using a point of inflection of a large PSF (something like that?)
 - [ ] Implement assertion to confirm number of y samples and decimation factor follow critical formula from adaptive filtering approach paper
 - [ ] June 9 - Andrew suggests updating my decimation routine to be more dynamic, supporting powers of 2 or even arbitrary amounts, not sure how hard that is? He suggests experimenting with variable decimation and using a fixed optimal beta, fixed optimal sigma, and fixed PSF support (one of the higher end supports which seem stable, say 51? Could probably even just do the full PSF of 99x99 realistically

---

#### Notes:

- If one takes the $\beta S^T S$ portion of $A$ and multiplies it by the solved $X$;  assuming the result is not normalised, one can observe the contribution of beta-influenced laplacian sharpening towards $B$. It appears to float around 10% of the second portion of $A$ (the decimated PSF) multiplied by the solved $X$. Taking the sum of these two separate portions of $AX$ allow one to produce $B$. It appears that the laplacian sharpening portion of $A$ does meaningfully contribute to help find a reasonable solution to $X$, and doesnt appear to be dominating (i.e., drowning out) the decimated PSF portion of $A$.
- Essentially, we dont want the beta-driven laplacian to dominate the sharpened PSF, otherwise you end up with effectively an overly smooth image $X$, and the technique becomes less useful / less impressive. We want to solve for $X$, such that beta helps us resolve a more accurate image $X$ which isnt just noise (a lack of structure, looks like static), and also doesnt look too smooth. If solving was less useful because we required large beta terms, then we might as well just average our set of $Y$ low-resolution images instead since its computationally cheaper?
 - Implement nifty gridder as a jupyter notebook - Andrew wants to see an image of true x, solved x, and nifty x
 - Andrew suggestions that in the future, a good method for analysing the effectiveness of this technique would be to compare the extractable sources and compare against a sky model (possibly Fred Dulwich has this?). It seems RRMSE is "misleading", in the sense that images produced by this technique arent so noisy, sources appear more gaussian-like and less like the dirty beam.
 - Furthermore, I should have a look at how this technique compares against the direct image model, versus an image produced by means of gridding (i.e., w-projection, nifty).
---

In [1]:
%matplotlib inline

import numpy as np
# import cupy as cp
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from scipy.stats import norm
from scipy.sparse import csr_matrix, bsr_matrix
from scipy.sparse.linalg import spsolve
from scipy.linalg import solve
from skimage import data, io, color
from skimage.transform import resize

plt.rcParams['figure.figsize'] = [10, 10]

def show_image(image, title, flip_x_axis=False):
    if flip_x_axis:
        image = np.fliplr(image)
    plt.imshow(image, cmap=plt.get_cmap("gray"))
    plt.title(title)
    plt.colorbar()
    plt.show()
    
def normalise(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def rrmse(observed, ideal, decimal=6):
    return "{:.{}f}".format(np.sqrt((1 / observed.shape[0]**2) * np.sum((observed-ideal)**2) / np.sum(ideal**2)) * 100.0, decimal)

def laplacian_of_gaussian(x, y, sigma):
    p = (x**2.0 + y**2.0) / 2.0 * sigma**2.0
    return -(1.0 / (np.pi * sigma**4.0)) * (1.0 - p) * np.exp(-p)

# def decimation_matrix(l, m):
#     d_matrix = np.zeros((m**2, l**2), dtype=np.float32)

#     tile = np.repeat((1, 0, 1), (2, l - 2, 2)) # assuming taking 2 neighbours per dimension
#     t_len = tile.shape[0]
#     d = l // m
#     r_offset = m**2 // 2
#     c_offset = l**2 // 2

#     for p in np.arange(l//4): # divide by 4 as 4 neighbours total
#         p_offset = p * l
#         for q in np.arange(m):
#             d_matrix[q+ p_offset//2, q*d + p_offset*2 : q*d+t_len + p_offset*2] = tile # top-left quadrant
#             d_matrix[q+r_offset + p_offset//2, q*d+c_offset + p_offset*2: q*d+t_len+c_offset + p_offset*2] = tile # bottom-right quadrant
#     return d_matrix

# Generates a sparse decimation matrix using the Block Sparse Row matrix format (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.bsr_matrix.html#scipy-sparse-bsr-matrix)
def decimation_matrix(original_dim, downsample_factor):
    
    if original_dim % downsample_factor != 0:
        raise ValueError(f"Downsample factor {downsample_factor} is not a valid factor of your matrix dimension {original_dim}.")
    if downsample_factor == original_dim:
        raise ValueError(f"Downsample factor {downsample_factor} cannot be the same as your matrix dimension {original_dim}.")
    if downsample_factor == 1: # effectively, no downsampling
        return np.identity(original_dim**2)
    # Otherwise assumed you want to downsample by a valid factor of original_dim...
    
    sampling_regions_per_dim = original_dim // downsample_factor
    samples_per_region_dim = downsample_factor
    # print(f"Sampling regions per dimension: {sampling_regions_per_dim}")
    # print(f"Samples per dimension: {samples_per_region_dim}")
    non_zero_entries = sampling_regions_per_dim**2 * samples_per_region_dim**2
    # print(f"Non-zero entries: {non_zero_entries}")
    
    rows = np.zeros(non_zero_entries, dtype=np.uintc)   # stores row indices for non-zero compressed sparse matrix entries
    cols = np.zeros(non_zero_entries, dtype=np.uintc)   # stores col indices for non-zero compressed sparse matrix entries
    vals = np.ones(non_zero_entries, dtype=np.float32)  # stores element value at [row, col] for non-zero entries
    
    # Generates linear x,y index strides for downsampling
    sample_stride_1D = np.arange(0, original_dim, downsample_factor)
    # print(sample_stride_1D)
    mesh = np.array(np.meshgrid(sample_stride_1D, sample_stride_1D))
    sample_strides_2D = mesh.T.reshape(-1, 2)
  
    neighbour_strides_1D = np.arange(samples_per_region_dim)
    neighbour_mesh = np.array(np.meshgrid(neighbour_strides_1D, neighbour_strides_1D))

    for index in np.arange(sample_strides_2D.shape[0]):
        neighbour_coords = neighbour_mesh.T.reshape(-1, 2) + sample_strides_2D[index] # generates (row, col) index pair for the nxn neighbours of each sampling point in sample_strides_2D
        neighbour_coords[:, 0] *= original_dim # scale y coord by high-resolution image dim to enable row striding (due to column-vector matrix flattening)
        neighbour_coords = np.sum(neighbour_coords, axis=1) # combine x and y coord into single array index
        rows[index * neighbour_coords.shape[0] : (index + 1) * neighbour_coords.shape[0]] = index
        cols[index * neighbour_coords.shape[0] : (index + 1) * neighbour_coords.shape[0]] = neighbour_coords
        
    return bsr_matrix((vals, (rows, cols)))

# Generates a sparse convolution matrix using the Block Sparse Row matrix format (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.bsr_matrix.html#scipy-sparse-bsr-matrix)
def convolution_matrix(l, kernel):
    kernel_full_supp = kernel.shape[0]
    # print(f"Kernel full supp: {kernel_full_supp}")
    kernel_half_supp = (kernel_full_supp - 1) // 2
    # print(f"Kernel half supp: {kernel_half_supp}")
    non_zero_entries = kernel_full_supp**2 * l**2
    # print(f"Total non-zero entries: {non_zero_entries}")
    # print(f"Or in MB: {non_zero_entries / 10**6}")
    rows = np.arange(non_zero_entries, dtype=np.uintc)
    cols = np.arange(non_zero_entries, dtype=np.intc)
    vals = np.arange(non_zero_entries, dtype=np.float32)

    print(f"Rows: {rows.size * rows.itemsize / 10**6} MB")
    print(f"Cols: {cols.size * cols.itemsize / 10**6} MB")
    print(f"Vals: {vals.size * vals.itemsize / 10**6} MB")
    
#     print(rows)
#     print(cols)
#     print(vals)
    
#     # for each flattened "convolution" row in matrix
#     for m_row in np.arange(l**2):
#         # print(f"On m_row {m_row}")
#         # map flattened m_row to 2d row/col  
#         row, col = (m_row // l, m_row % l)
#         # map kernel to neighbouring indices of row/col, from centre of kernel
#         neighbour_rows =  np.arange(-(kernel_half_supp), kernel_half_supp+1) + row
#         neighbour_cols = np.arange(-(kernel_half_supp), kernel_half_supp+1) + col
#         mesh = np.array(np.meshgrid(neighbour_rows, neighbour_cols))
#         neighbour_indices = mesh.T.reshape(-1, 2)
#         # zero out in kernel, any neighbours which have negative index or greater than l
#         bad_rows = np.concatenate((np.where(neighbour_indices[:, 0] < 0)[0], np.where(neighbour_indices[:, 0] >= l)[0]))
#         bad_cols = np.concatenate((np.where(neighbour_indices[:, 1] < 0)[0], np.where(neighbour_indices[:, 1] >= l)[0]))
#         # get a set of distinct neighbours to be zeroed out (in case of duplicate entries where a bad row is also a bad column)
#         bad_neighbours = np.unique(np.concatenate((bad_rows, bad_cols)))
#         # flatten 2d kernel and zero out invalid entries
#         kernel = kernel.flatten()
#         kernel[bad_neighbours] = 0.0
#         # determine where to copy over row/col/kernel entries
#         start = m_row * kernel_full_supp**2
#         end = (m_row + 1) * kernel_full_supp**2
#         col_strides = np.repeat((np.arange(-kernel_half_supp, kernel_half_supp + 1)), (np.repeat(kernel_full_supp, kernel_full_supp))) * (l - kernel_full_supp)
#         # populating row, strided col indices and flattened kernel slices
#         rows[start:end] = m_row
#         cols[start:end] = (np.arange(m_row, m_row + kernel_full_supp**2) - kernel_full_supp**2 // 2 + col_strides)
#         vals[start:end] = kernel

#     # strip away any entries in rows/cols/vals where cols < 0, cols >= l**2, or where val is zero
#     out_of_bounds_or_zero = np.concatenate((np.where(cols < 0), np.where(cols >= l**2), np.where(vals == 0.0)), axis=None)
#     rows = np.delete(rows, out_of_bounds_or_zero)
#     cols = np.delete(cols, out_of_bounds_or_zero)
#     vals = np.delete(vals, out_of_bounds_or_zero)
    
    print("BUILDING UP BSR MATRIX RIGHT NOW...")
    pass
    # return csr_matrix((vals, (rows, cols)))

#### Configuration and data set up...

In [2]:
timesteps = 30 # total timesteps
timesteps_per_y = 1
l = 400
m = 100
n = timesteps // timesteps_per_y
w = np.ones(n)
β = 8.0

# False = use static 3x3 laplacian kernel
# True = use customisable laplacian of gaussian kernel
use_laplacian_of_gaussian = True

# all time steps direct image
# filename = "../data/direct_image_ts_0_29.bin"
# filename = "../data/direct_image_ts_0_29_800x800.bin"
filename = "../datasets/gleam_small/images/direct_image_ts_0_29_400x400.bin"
x_true = np.fromfile(filename, dtype=np.float32)
x_true = np.reshape(x_true, (l, l))
# x_true = resize(x_true.reshape(800, 800), (l, l), anti_aliasing=False, order=1)
x_true = normalise(x_true)
# show_image(x_true, "True X")

# x_orig = np.fromfile("../data/direct_image_ts_0_29.bin", dtype=np.float32)
# show_image(normalise(x_orig.reshape(l, l)), "Original True X (100^2)")

# all time steps direct psf
# filename = "../data/direct_psf_ts_0_29.bin"
# filename = "../data/direct_psf_ts_0_29_800x800.bin"
# x_psf = np.fromfile(filename, dtype=np.float32)
# x_psf = resize(x_psf.reshape(800, 800), (l, l), anti_aliasing=False, order=1)
# x_psf = x_psf.reshape(l, l)[1:, 1:] # trim row 0 and all column 0 (to ensure odd dimensions with the peak in the center)
# x_psf /= np.max(x_psf)
# show_image(x_psf, "PSF (800^2 averaged to 100^2)")

# filename = "../data/direct_psf_ts_0_29_800x800.bin"
filename = "../datasets/gleam_small/images/direct_psf_ts_0_29_400x400.bin"
x_psf = np.fromfile(filename, dtype=np.float32).reshape(400, 400)[1:, 1:]
# show_image(x_psf, "Untrimmed PSF")
x_psf = resize(x_psf, (l-1, l-1), anti_aliasing=False, order=1)
x_psf = np.pad(x_psf, ((1, 0), (1, 0))) # pad with new 0th row/col to ensure trimming from centre
# show_image(x_psf, "Untrimmed PSF - RESIZED")
# plt.plot(x_psf[x_psf.shape[0]//2])
# plt.show()

trim_half_len = 50
psf_min = l//2 - (trim_half_len - 1)
psf_max = l//2 + trim_half_len
psf_support = psf_max - psf_min
# print(psf_support)
# x_psf = x_psf.reshape(l, l)[psf_min:psf_max, psf_min:psf_max]
x_psf_trimmed = x_psf.copy()[psf_min:psf_max, psf_min:psf_max]
x_psf_trimmed /= np.sum(x_psf_trimmed)
# print(np.sum(x_psf_trimmed))
# plt.plot(x_psf[x_psf.shape[0]//2])
# plt.show()

# print(f"PSF Shape: {x_psf_trimmed.shape[0]}")
# show_image(x_psf_trimmed, "PSF")

print(f"l^2:                      {l**2}")
print(f"(psf_support - 2) * m**2: {(psf_support - 2) * m**2}")
print(f"n * m**2:                 {n * m**2}")

# Determine whether we have the right criteria to perform super-resolution imaging to obtain X
if l**2 <= np.minimum((psf_support - 2) * m**2, n * m**2):
    print("Super-resolution imaging technically possible, continuing...\n")
else:
    raise ValueError("High-resolution image not possible to reproduce from your set up, review your configuration against this if statement conditional.\n")

# Storing all low-res images as layered stack
y = np.zeros((n, m, m))

# batched time steps direct images
for i in np.arange(n):
    timestep_range_start = i * timesteps_per_y
    timestep_range_end = timestep_range_start + timesteps_per_y
    filename = f"../datasets/gleam_small/images/direct_image_ts_{timestep_range_start}_{timestep_range_end - 1}_{m}x{m}.bin"
    y[i] = np.fromfile(filename, dtype=np.float32).reshape(m, m)
    y[i] = normalise(y[i])
    # show_image(y[i], f"$Y {i}$")
    
# batched time steps point spread functions
# for i in np.arange(N):
#     start = i * timesteps_per_y
#     end = start + timesteps_per_y - 1
#     filename = "../data/direct_psf_ts_%d_%d.bin" % (start, end)
#     Y_i_psf = np.fromfile(filename, dtype=np.float32)
#     Y_i_psf = Y_i_psf.reshape(L, L)
#     # show_image(Y_i_psf, "$Y_{%d}$ PSF" % i)

# Decimation matrix
d = decimation_matrix(l, 4)
# d_numpy_size_bytes = d.size * d.itemsize
# print(f"D matrix uncompressed sparse: {d_numpy_size_bytes / 10**6} MB")
# d = bsr_matrix(d)
d_csr_matrix_bytes = d.data.nbytes + d.indptr.nbytes + d.indices.nbytes
print(f"D matrix compressed sparse: {d_csr_matrix_bytes / (10**6)} MB")
# print(f"D matrix reduced by {100 - (d_csr_matrix_bytes / d_numpy_size_bytes * 100)}%\n")

l^2:                      160000
(psf_support - 2) * m**2: 970000
n * m**2:                 300000
Super-resolution imaging technically possible, continuing...

D matrix compressed sparse: 1.320004 MB


In [3]:
# Blur matrix (psf)
convolution_matrix(l, x_psf_trimmed)
# print("H returned...")
# h_numpy_size_bytes = h.size * h.itemsize
# print(f"H matrix uncompressed sparse: {h_numpy_size_bytes / 10**6} MB")
# h = bsr_matrix(h)
# h_csr_matrix_bytes = h.data.nbytes + h.indptr.nbytes + h.indices.nbytes
# print(f"H matrix compressed sparse: {h_csr_matrix_bytes / (10**9)} GB")
# print(f"H matrix reduced by {100 - (h_csr_matrix_bytes / h_numpy_size_bytes * 100)}%\n")
# show_image(h.todense(), "H")

Rows: 6272.64 MB
Cols: 6272.64 MB
Vals: 6272.64 MB
BUILDING UP BSR MATRIX RIGHT NOW...


In [ ]:
# Sharpening matrix (laplacian)
if use_laplacian_of_gaussian:
    psf_central_support = 60 # 15 # num pixels around centre of PSF which represent approx 1/3 of the curve, maybe a bit bigger for padding
    downsampling = np.linspace(-(psf_central_support-1)//2, (psf_central_support-1)//2, num=psf_central_support)
    lap_of_gauss = np.zeros((downsampling.shape[0], downsampling.shape[0]), np.float32)
    lap_gauss_σ = 2.205128205 # need to play around with this to get similar structure to true laplacian 3x3
    for i in np.arange(downsampling.shape[0]):
        for j in np.arange(downsampling.shape[0]):
            lap_of_gauss[i][j] = -laplacian_of_gaussian(downsampling[i], downsampling[j], lap_gauss_σ)

    lap_of_gauss /= np.max(lap_of_gauss)

    # plt.plot(lap_of_gauss[lap_of_gauss.shape[0]//2])
    # plt.show()

    s = convolution_matrix(l, lap_of_gauss)
    β *= 4.0
    # print(f"Laplacian Shape: {lap_of_gauss.shape[0]}")
else:
    laplacian = np.array([[0, -1,  0], [-1,  4, -1], [0, -1,  0]], dtype=np.float32)
    s = convolution_matrix(l, laplacian)

s_numpy_size_bytes = s.size * s.itemsize
print(f"S matrix uncompressed sparse: {s_numpy_size_bytes / 10**6} MB")
s = bsr_matrix(s)
s_csr_matrix_bytes = s.data.nbytes + s.indptr.nbytes + s.indices.nbytes
print(f"S matrix compressed sparse: {s_csr_matrix_bytes / (10**6)} MB")
print(f"S matrix reduced by {100 - (s_csr_matrix_bytes / s_numpy_size_bytes * 100)}%\n")
    
dh = d @ h
    
# Notes to self: seems like lap_gauss_σ sits around 1.41 for this example, any higher or lower results in worse RRMSE. Changing the psf_central_support makes no difference, and taking more samples (i.e. num)
# just means that lap_gauss_σ is multiplied by the factor of difference (i.e., double the num samples needs double the lap_gauss_σ). Also found that beta is best multiplied by 4.0, not 16 (the square).
# Not sure what to do from here... seems like the laplacian of gaussian doesnt really serve any beneficial purpose...

#### Setting up the right hand side of the equation $AX = B$, where $B = \sum\limits_{i=1}^N (w_iH^TD^TY_i)$

In [ ]:
b = np.zeros(l**2, dtype=np.float32)

for i in np.arange(n):
    b += w[i] * bsr_matrix.transpose(dh) @ y[i].flatten()
    
# show_image(normalise(b.reshape(l, l)), "B")

In [ ]:
print((b.size * b.itemsize) / 10**6)

#### Setting up the right hand side of the equation $AX=B$, where $A = \begin{bmatrix}
  \beta S^T S + (\sum\limits_{i=1}^N w_i) H^TD^TDH
\end{bmatrix}$

In [ ]:
%%time
# lhs = β * np.matmul(s.T, s)
# rhs = (h.T @ d.T @ d @ h) * np.sum(w)
a = (β * s @ bsr_matrix.transpose(s)) + (bsr_matrix.transpose(dh) @ dh * np.sum(w))

# a = bsr_matrix.toarray(a)
# show_image(a.reshape(l**2, l**2), "A")

In [ ]:
print(type(a))
print((a.data.nbytes + a.indptr.nbytes + a.indices.nbytes) / 10**6)

#### Now solve for $X$...

In [ ]:
%%time
# Solving via CPU and numpy...
# x = np.linalg.solve(a, b)
# x = spsolve(a, b) # around 2 mins 10 seconds for this routine
x = solve(a.todense(), b) # wow, about 5 seconds...
x = x.reshape(l, l)
# x = normalise(x)

In [ ]:
# # Solving via GPU and cupy
# a_gpu = cp.asarray(a)
# b_gpu = cp.asarray(b)
# solution = cp.linalg.solve(a_gpu, b_gpu)
# x = cp.asnumpy(solution)
# x = x.reshape(100, 100)

# # Dealloc cuda mem
# mempool = cp.get_default_memory_pool()
# mempool.free_all_blocks()

In [ ]:
# b_true = np.matmul(normalise(a), x_true.flatten())
# x_accurate = np.linalg.solve(a, b_true)
# x_accurate = x_accurate.reshape(100, 100)

# show_image(normalise(x), "Solved X")
# print(f"RRMSE: B and True B -> {rrmse(normalise(b.reshape(l, l)), normalise(b_true.reshape(l, l)))}")
# print(f"L2 between B and True X: {np.sqrt(np.sum((normalise(b.reshape(l, l))-normalise(x))**2))}")
# show_image(normalise(b.reshape(l, l)), "B")
# show_image(normalise(b_true.reshape(l, l)), "True B")

# print(f"RRMSE: X and Accurate X -> {rrmse(normalise(x), normalise(x_accurate))}")
# print(f"RRMSE: Accurate X and True X -> {rrmse(normalise(x_accurate), normalise(x_true))}")
print(f"Beta: {β}")
print(f"RRMSE: Solved X and True X -> {rrmse(normalise(x), normalise(x_true))}")
print(f"True X peak location: {np.unravel_index(np.argmax(x_true, axis=None), x_true.shape)}")
print(f"Solved X peak location: {np.unravel_index(np.argmax(x, axis=None), x.shape)}")

# plt.plot(x_true[93, :], label="True X")
# plt.plot(x[93, :], label="Solved X")
# plt.legend()
# plt.show()

# Multiply A left hand side (beta and sharpening matrices) by X
# a_laplacian = lhs # np.matmul(lhs, x.flatten()).reshape(l, l)
# show_image((a_laplacian), "A-laplacian", flip_x_axis=True)
# a_lap_l1 = np.max(np.sum(np.absolute(a_laplacian), axis=0))
# a_lap_linf = np.max(np.sum(np.absolute(a_laplacian), axis=1))
# print(f"A_laplacian l1 norm: {a_lap_l1}")
# print(f"A_laplacian linf norm: {a_lap_linf}")

# a_deci_blur = rhs # np.matmul(rhs, x.flatten()).reshape(l, l)
# show_image((a_deci_blur), "A-deci-blur", flip_x_axis=True)
# a_dec_l1 = np.max(np.sum(np.absolute(a_deci_blur), axis=0))
# a_dec_linf = np.max(np.sum(np.absolute(a_deci_blur), axis=1))
# print(f"A_deci_blur l1 norm: {a_dec_l1}")
# print(f"A_deci_blur linf norm: {a_dec_linf}")

# print(f"A_deci_blur / A_laplacian l1 ratio: {a_dec_l1 / a_lap_l1}")
# print(f"A_deci_blur / A_laplacian linf ratio: {a_dec_linf / a_lap_linf}")

show_image(normalise(x), "Solved X", flip_x_axis=True)
# show_image(normalise(x_accurate[10:l-11, 10:l-11]), "Solved X using True X (IDFT)")
# show_image(normalise(x_accurate), "Solved X using True X (IDFT)")
show_image(normalise(x_true), "True X", flip_x_axis=True)
# show_image(normalise(a_laplacian + a_deci_blur), "A parts summed to B", flip_x_axis=True)

### Experimenting with different high/low resolution image sizes with variable number of low-resolution images

- Dataset: gleam small
- High-resolution image size: generated at 800x800, downsampled to 100x100 for solving in reasonable time
- PSF support: 51x51

| L   | M  | N  | Beta        | Beta Scaled | Laplacian or LoG | LoG Sigma   | RRMSE (%) | 
| -   | -  | -  | -           | -           | -                | -           | -         | 
| 100 | 50 | 6  | 0.217189985 | N/A         | Laplacian        | N/A         | 1.144697  |
| 100 | 50 | 6  | 0.217189985 | 0.86875994  | LoG              | 2.205128205 | 0.763645  |
| -   | -  | -  | -           | -           | -                | -           | -         | 
| 100 | 25 | 30 | 0.217189985 | N/A         | Laplacian        | N/A         | 2.763222  |
| 100 | 25 | 30 | 0.217189985 | 0.86875994  | LoG              | 2.205128205 | 2.530623  |
| 100 | 25 | 30 | 0.4         | 1.6         | LoG              | 2.205128205 | 2.221223  |
| 100 | 25 | 30 | 0.6         | 2.4         | LoG              | 2.205128205 | 1.990213  |
| 100 | 25 | 30 | 1.0         | 4.0         | LoG              | 2.205128205 | 1.636430  |
| 100 | 25 | 30 | 1.25        | 5.0         | LoG              | 2.205128205 | 1.555625  |
| 100 | 25 | 30 | 1.5         | 6.0         | LoG              | 2.205128205 | 1.524417  |
| 100 | 25 | 30 | 2.0         | 8.0         | LoG              | 2.205128205 | 1.460980  |
| 100 | 25 | 30 | 3.0         | 12.0        | LoG              | 2.205128205 | 1.336314  |
| 100 | 25 | 30 | 4.0         | 16.0        | LoG              | 2.205128205 | 1.229604  |
| 100 | 25 | 30 | 5.0         | 20.0        | LoG              | 2.205128205 | 1.194185  |
| 100 | 25 | 30 | 8.0         | 32.0        | LoG              | 2.205128205 | 1.086680  |

In [ ]:
x.tofile(f"../datasets/gleam_small/images/solved_image_{x.shape[0]}x{x.shape[1]}.bin")